There is that thing is haskell in which a function can automatically select an overloaded variant based on the annotation of its caller

eg:
```haskell
f :: Int -> Maybe Int
f x = pure (x * 2)
```

In [1]:
import inspect
from dataclasses import dataclass, astuple
from functools import wraps
from typing import Any

In [2]:
class Polymorphic:
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        dataclass(frozen=True, eq=True, order=True, repr=False)(cls)

    def __repr__(self):
        cls = self.__class__.__name__
        args = ', '.join(map(repr, astuple(self)))
        return f'{cls}({args})'

    def to(self, type):
        return getattr(type, self.__class__.__name__)(*astuple(self))

In [3]:
class pure(Polymorphic):
    x : Any

In [4]:
def resolve_return(f):
    @wraps(f)
    def _resolve_return(*args, **kwargs):
        result = f(*args, **kwargs)
        if not isinstance(result, Polymorphic):
            return result 
        return result.to(inspect.signature(f).return_annotation(result))
    return _resolve_return


@dataclass
class Box:
    x : int

    @staticmethod
    def pure(x):
        return Box(x)


@resolve_return
def bar() -> Box:
    return pure(10)

In [5]:
bar()

Box(x=10)